# Com algoritmo k-NN

```
pip install scikit-surprise
```
```
pip install sqlalchemy
```
```
pip install psycopg
```
```
sudo apt-get install libpq-dev
```
```
pip install psycopg2
```
```
pip install -U psycopg2==2.9.1
```

## Indicações com base na idade e tipo de culinária

In [30]:
import psycopg2 as pg
import pandas as pd
from sqlalchemy import create_engine

# Substitua pelas suas próprias informações de conexão
host = "bfzaywtx287rqy6l6s8m-postgresql.services.clever-cloud.com"
dbname = "bfzaywtx287rqy6l6s8m"
user = "ujhrqenft29mcpu7xvwx"
password = "4xdZfWCSz5DyTldDnuoxqlMgWganMb"

conn = pg.connect(
    host=host,
    dbname=dbname,
    user=user,
    password=password
)

# Crie uma URI de conexão para o PostgreSQL
conn_str = f"postgresql://{user}:{password}@{host}/{dbname}"

# Crie uma conexão com o SQLAlchemy
engine = create_engine(conn_str)
query_cousine = "select u.id as user_id, u.age as age, rest.cousine_type from requests r inner join restaurants rest on r.restaurant_id = rest.id inner join users u on r.user_id = u.id order by r.id;"


query_food = "select u.id as user_id, m.food_name, u.age as age from requests r inner join menus m  on r.food_id = m.id inner join users u on r.user_id = u.id order by r.id;"

df_sql_cousine = pd.read_sql_query(query_cousine, con=engine)
df_sql_food = pd.read_sql_query(query_food, con=engine)


conn.close()

In [31]:
from surprise import Dataset, Reader, KNNBasic

#data = pd.read_csv('/home/viniciuspolo/developer/dsm/6_semestre/pi6/preferenceByAge.csv', sep=';')
data = df_sql_cousine

data.loc[data['age'] == '18-', 'age'] = 1
data.loc[data['age'] == '18 e 25', 'age'] = 2
data.loc[data['age'] == '25 e 30', 'age'] = 3
data.loc[data['age'] == '30 e 40', 'age'] = 4
data.loc[data['age'] == '40 e 60', 'age'] = 5
data.loc[data['age'] == '60+', 'age'] = 6

# Crie um DataFrame Pandas com os dados
df = pd.DataFrame(data)

# Defina o formato do leitor
reader = Reader(rating_scale=(1, 6))

# Carregue o conjunto de dados
data = Dataset.load_from_df(df[['user_id', 'cousine_type', 'age']], reader)

# Construa o conjunto de treinamento
trainset = data.build_full_trainset()

# Crie o modelo KNN
sim_options = {
    'name': 'cosine',
    'user_based': False
}
model = KNNBasic(sim_options=sim_options)

model.fit(trainset)

# ID do filme para o qual você deseja obter recomendações
cousine_type = "CHURRASCO"

# Obtendo as recomendações para o filme
age_inner_id = trainset.to_inner_iid(cousine_type)
neighbors = model.get_neighbors(age_inner_id, k=1)

# Mapeando os IDs internos de volta para os IDs reais dos filmes
neighbors = [trainset.to_raw_iid(inner_id) for inner_id in neighbors]

print(f"Quem pede culinária {cousine_type} Pode gostar também de:")
for neighbor in neighbors:
    print(neighbor)


Computing the cosine similarity matrix...
Done computing similarity matrix.
Quem pede culinária CHURRASCO Pode gostar também de:
JAPONESA


## Indicações com base nos prato e idade

In [32]:
from surprise import Dataset, Reader, KNNBasic

#data = pd.read_csv('/home/viniciuspolo/developer/dsm/6_semestre/pi6/requestsByAgeAndGender.csv', sep=';')
data = df_sql_food

data.loc[data['age'] == '18-', 'age'] = 1
data.loc[data['age'] == '18 e 25', 'age'] = 2
data.loc[data['age'] == '25 e 30', 'age'] = 3
data.loc[data['age'] == '30 e 40', 'age'] = 4
data.loc[data['age'] == '40 e 60', 'age'] = 5
data.loc[data['age'] == '60+', 'age'] = 6

# Crie um DataFrame Pandas com os dados
df = pd.DataFrame(data)

# Defina o formato do leitor
reader = Reader(rating_scale=(1, 6))

# Carregue o conjunto de dados
data = Dataset.load_from_df(df[['user_id', 'food_name', 'age']], reader)

# Construa o conjunto de treinamento
trainset = data.build_full_trainset()

# Crie o modelo KNN
sim_options = {
    'name': 'cosine',
    'user_based': False
}
model = KNNBasic(sim_options=sim_options)

model.fit(trainset)

# ID do filme para o qual você deseja obter recomendações
food_name = "Picanha"

# Obtendo as recomendações para o filme
age_inner_id_by_food = trainset.to_inner_iid(food_name)
neighborsByfood = model.get_neighbors(age_inner_id_by_food, k=3)

# Mapeando os IDs internos de volta para os IDs reais dos filmes
neighborsByfood = [trainset.to_raw_iid(inner_id) for inner_id in neighborsByfood]

print(f"Quem pede {food_name} Pode gostar também de:")
for neighbor in neighborsByfood:
    print(neighbor)


Computing the cosine similarity matrix...
Done computing similarity matrix.
Quem pede Picanha Pode gostar também de:
Combinado Sushi Salmão
Saquê
Teppanyaki


In [33]:
print(f"Hum, você escolheu um delicioso {food_name} Acreditamos que pode gostar também de:")
for neighbor in neighborsByfood:
    print(neighbor)
print(f"\nE já que pediu um prato do tipo {cousine_type} talvez também queira explorar a culinária")
for neighbor in neighbors:
    print(neighbor)

Hum, você escolheu um delicioso Picanha Acreditamos que pode gostar também de:
Combinado Sushi Salmão
Saquê
Teppanyaki

E já que pediu um prato do tipo CHURRASCO talvez também queira explorar a culinária
JAPONESA
